In [16]:
# notebook to calculate implicit/explicit metrics mentioned in paper
# The metrics definition has been taken from https://github.com/WING-NUS/JD2Skills-BERT-XMLC
# inputs are just the jd, true labels and predictions for testing data (you can find them in the model folder when you run training + testing)

In [17]:
# load libraries
import pandas as pd
import numpy as np


### Load data

In [21]:
# Load predictions

with open('/home/jsk/skill-prediction/data/JOBS-Stack/models/GraphXMLModelNR_job-stack-jobs/predictions_20','r') as f:
    data = f.readlines()
pred_data = [(x.split('\t')[0]) for x in data]

# load ground truth
with open('/home/jsk/skill-prediction/data/JOBS-Stack/models/GraphXMLModelNR_job-stack-jobs/true_20','r') as f:
    data = f.readlines()
gold_data = [(x.split('\t')[0]) for x in data]

# load job descriptions
with open('/home/jsk/skill-prediction/data/JOBS-Stack/tst_X.txt') as f:
    data = f.readlines()

jd = [x.split('\n')[0] for x in data]



In [14]:
# pd.set_option('display.max_colwidth', None)
# df_show= df[['True Labels','COLING', 'latest']]
# df_show.head()

In [29]:
# temporary accomodation
predictions = [eval(x) for x in data_pred][:2097]
gold_data = [eval(x) for x in gold_data][:2097]


In [30]:
assert len(gold_data) == len(predictions) == len(jd)

2097


# Class

In [9]:
class Metric:
    def __init__(self):
#         self.epsilon = 1.0e-4
        self.epsilon = 1
        pass

    def __call__(self, outputs, target):
        raise NotImplementedError

    def reset(self):
        raise NotImplementedError

    def value(self):
        raise NotImplementedError

    def name(self):
        raise NotImplementedError

class Accuracy(Metric):
    def __init__(self,topK):
        super(Accuracy,self).__init__()
        self.topK = topK
        self.reset()

    def __call__(self, logits, target):
        _, pred = logits.topk(self.topK, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))
        self.correct_k = correct[:self.topK].view(-1).float().sum(0)
        self.total = target.size(0)

    def reset(self):
        self.correct_k = 0
        self.total = 0

    def value(self):
        return float(self.correct_k)  / self.total

    def name(self):
        return 'accuracy'


class AccuracyThresh(Metric):
    def __init__(self,thresh = 0.5):
        super(AccuracyThresh,self).__init__()
        self.thresh = thresh
        self.reset()

    def __call__(self, logits, target):
        self.y_pred = logits.sigmoid()
        self.y_true = target

    def reset(self):
        self.correct_k = 0
        self.total = 0

    def value(self):
        data_size = self.y_pred.size(0)
        acc = np.mean(((self.y_pred>self.thresh).type(torch.DoubleTensor)==self.y_true.type(torch.DoubleTensor)).float().cpu().numpy(), axis=1).sum()
        return acc / data_size

    def name(self):
        return 'accuracy'

class MRR(Metric):
    def __init__(self):
        super(MRR,self).__init__()
        self.rr = []

    def __call__(self, y_pred, y_true):
        self.y_pred = y_pred
        self.y_true = y_true

        for i in range(len(self.y_pred)):
            y_t = np.where(self.y_true[i]==1)[0]
            y_t_len = len(y_t)
            y_p_index = np.flip(np.argsort(self.y_pred[i]),0)
            for i in range(len(y_p_index)):
                if y_p_index[i] in y_t:
                    self.rr.append(1/float(i+1))
                    break

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.rr))*100

    def name(self):
        return 'mrr'

    def show(self):
        print("MRR: {}".format(self.value()))

class NDCG(Metric):
    def __init__(self):
        super(NDCG,self).__init__()
        self.NDCG = []
        self.kvals = [5,10,30,50,100]
        self.reset()

    def __call__(self, logits, target):
        self.y_pred = logits.sigmoid().cpu().numpy()
        self.y_true = target.cpu().numpy()
        
        for i in range(len(self.y_pred)):
            y_t = np.where(self.y_true[i]==1)[0]
            y_t_len = len(y_t)
            y_p_index = np.flip(np.argsort(self.y_pred[i]),0)
            idcg = np.sum([1.0/np.log2(x+2) for x in range(y_t_len)])
            ndcg = []
            for k in self.kvals:
                dcg = 0 
                for i in range(0,k):
                    if y_p_index[i] in y_t:
                        dcg = dcg + 1.0/np.log2(i+2)
                ndcg.append(dcg/idcg)
            self.NDCG.append(ndcg)

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.NDCG),axis=0)*100

    def name(self):
        return 'ndcg'

    def show(self):
        idx = 0
        result = self.value()
        for k in self.kvals:
            print("NDCG@{}: {}".format(k,result[idx]))
            idx += 1
        return result

class Recall(Metric):
    def __init__(self):
        super(Recall,self).__init__()
        self.Recall = []
        self.kvals = [5,10,20,30,50,100]
        self.reset()

    def __call__(self, logits, target):
        self.y_pred = logits.sigmoid().cpu().numpy()        
        self.y_true = target.cpu().numpy()
        for i in range(len(self.y_pred)):
            recall = []
            y_p_index = np.flip(np.argsort(self.y_pred[i]),0) # What is this doing, needs to sort highest probability
            y_t = np.where(self.y_true[i]==1)[0] # Returns index where condition true. an array 
            y_t_len = len(y_t)

            for k in self.kvals:
                correct = len(np.intersect1d(y_t,y_p_index[0:k]))
                recall.append(correct/(y_t_len+self.epsilon))
            self.Recall.append(recall)
        

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.Recall),axis=0)*100

    def name(self):
        return 'recall'
    
    def show(self):
        idx = 0
        result = self.value()
        for k in self.kvals:
            print("Recall@{}: {}".format(k,result[idx]))
            idx += 1
        return result




class Precision(Metric):
    def __init__(self):
        super(Precision,self).__init__()
        self.Precision = []
        self.kvals = [5,10,20,30,50,100]
        self.reset()
        self.count_ind=[]

    def __call__(self, logits, target):
        self.y_pred = logits.sigmoid().cpu().numpy()       
        self.y_true = target.cpu().numpy()
        for i in range(len(self.y_pred)):
            precision = []
            y_p_index = np.flip(np.argsort(self.y_pred[i]),0) # index where predicted values are true
            y_t = np.where(self.y_true[i]==1)[0] # indexes where actual truth labels 
            y_t_len = len(y_t)  # how many true labels

            # print("Number of skills actually correct :",y_t_len)
            # print("Number of skills we predict are correct for k 5,10,30,50,100:")

            for k in self.kvals:
                correct = len(np.intersect1d(y_t,y_p_index[0:k]))
                # print(correct)
                precision.append(correct/(k+self.epsilon))
            self.Precision.append(precision)
            # print("Indexes matched",np.intersect1d(y_t,y_p_index[0:k]))
        

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.Precision),axis=0)*100

    def name(self):
        return 'precision'
    
    def show(self):
        idx = 0
        result = self.value()
        for k in self.kvals:
            print("Precision@{}: {}".format(k,result[idx]))
            idx += 1
        return result
# implicit metrics are evaluated based on the top-20 predections 

class Implicit_Metrics(Metric):

    def __init__(self):
        super(Implicit_Metrics,self).__init__()
    
    def read_skills(self, path):
        skill_list = []
        with open(path, 'r') as f:
            for line in f.readlines():
                skill = line.replace('\n','')
                skill_list.append(skill)
        return skill_list

    def present_skills(self, job_description, skill_list):
        skills = []
        for skill in skill_list:
            if (' ' + skill.lower().strip() + ' ') in (' ' + job_description.lower().strip() + ' '):
                if skill not in skills:
                    skills.append(skill)
        return skills

    def generate_sentence(self, input_ids):
        input_ids = input_ids.tolist()
        sentence = ""
        for idx in input_ids:
            if self.i2w[idx][:2]=="##":
                sentence += self.i2w[idx][2:]
            else:
                sentence += " " + self.i2w[idx]
        return sentence.strip()
    
    def job_labels(self, label_indices):
        labels = []
        for idx in label_indices:
            labels.append(self.i2l[idx])
        return labels

    def intersection(self, lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3

class EIM(Implicit_Metrics):
    def __init__(self):
        super(EIM,self).__init__()
        self.EIM = []
        self.reset()

    def __call__(self, jd, y_pred, y_true):
        self.jd = jd
        self.y_pred = y_pred       
        self.y_true = y_true
        for i in range(len(self.y_pred)):
            self.EIM.append(self.eim(self.jd[i], self.y_pred[i], self.y_true[i]))
        
            
    def eim(self, jd, output, target):

        f = open('skill_list (1).txt').readlines()
        skill_lis = [x.split('\n')[0] for x in f]
#         print(skill_lis)
        
        explicit_skills = self.present_skills(jd, skill_lis) # skill in jd against all skills 

        if len(self.intersection(target, explicit_skills))==0:
            return 1
        return len(self.intersection(output, explicit_skills)) / (len(self.intersection(target, explicit_skills)))


    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.EIM))*100

    def name(self):
        return 'eim'
    
    def show(self):
        result = self.value()
        print("EIM: {}".format(result))

class RIIM(Implicit_Metrics):
    def __init__(self):
        super(RIIM,self).__init__()
        self.RIIM = []
        self.reset()

    def __call__(self, jd, y_pred, y_true):
        self.jd = jd
        self.y_pred = y_pred       
        self.y_true = y_true
        for i in range(len(self.y_pred)):
            self.RIIM.append(self.riim(self.jd[i], self.y_pred[i], self.y_true[i]))
        
    def riim(self, jd, output, target):
        f = open('skill_list (1).txt').readlines()
        skill_lis = [x.split('\n')[0] for x in f]
        
        explicit_skills = self.present_skills(jd, skill_lis)
        implicit_skills = [val for val in target if val not in explicit_skills]
        predicted_implicit_skills = [val for val in output if val in implicit_skills]
        return len(predicted_implicit_skills)/(len(implicit_skills) + self.epsilon)

    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.RIIM))*100

    def name(self):
        return 'riim'
    
    def show(self):
        result = self.value()
        print("RIIM: {}".format(result))

class REIM(Implicit_Metrics):
    def __init__(self):
        super(REIM,self).__init__()
        self.REIM = []
        self.reset()

    def __call__(self, jd, y_pred, y_true):
        self.jd = jd
        self.y_pred = y_pred       
        self.y_true = y_true
        for i in range(len(self.y_pred)):
            self.REIM.append(self.reim(self.jd[i], self.y_pred[i], self.y_true[i]))
        
            
    def reim(self, jd, output, target):

        f = open('skill_list (1).txt').readlines()
        skill_lis = [x.split('\n')[0] for x in f]
#         print(skill_lis)
        
        explicit_skills = self.present_skills(jd, skill_lis) # skill in jd against all skills 
        
        return len(self.intersection(output, explicit_skills)) / (len(explicit_skills)+self.epsilon)


    def reset(self):
        return

    def value(self):
        return np.mean(np.array(self.REIM))*100

    def name(self):
        return 'reim'
    
    def show(self):
        result = self.value()
        print("REIM: {}".format(result))


class AUC(Metric):
    '''
    AUC score
    micro:
            Calculate metrics globally by considering each element of the label
            indicator matrix as a label.
    macro:
            Calculate metrics for each label, and find their unweighted
            mean.  This does not take label imbalance into account.
    weighted:
            Calculate metrics for each label, and find their average, weighted
            by support (the number of true instances for each label).
    samples:
            Calculate metrics for each instance, and find their average.
    Example:
        >>> metric = AUC(**)
        >>> for epoch in range(epochs):
        >>>     metric.reset()
        >>>     for batch in batchs:
        >>>         logits = model()
        >>>         metric(logits,target)
        >>>         print(metric.name(),metric.value())
    '''

    def __init__(self,task_type = 'binary',average = 'binary'):
        super(AUC, self).__init__()

        assert task_type in ['binary','multiclass']
        assert average in ['binary','micro', 'macro', 'samples', 'weighted']

        self.task_type = task_type
        self.average = average

    def __call__(self,logits,target):
        if self.task_type == 'binary':
            self.y_prob = logits.sigmoid().data.cpu().numpy()
        else:
            self.y_prob = logits.softmax(-1).data.cpu().detach().numpy()
        self.y_true = target.cpu().numpy()

    def reset(self):
        self.y_prob = 0
        self.y_true = 0

    def value(self):
        auc = roc_auc_score(y_score=self.y_prob, y_true=self.y_true, average=self.average)
        return auc

    def name(self):
        return 'auc'

class F1Score(Metric):
    '''
    F1 Score
    binary:
            Only report results for the class specified by ``pos_label``.
            This is applicable only if targets (``y_{true,pred}``) are binary.
    micro:
            Calculate metrics globally by considering each element of the label
            indicator matrix as a label.
    macro:
            Calculate metrics for each label, and find their unweighted
            mean.  This does not take label imbalance into account.
    weighted:
            Calculate metrics for each label, and find their average, weighted
            by support (the number of true instances for each label).
    samples:
            Calculate metrics for each instance, and find their average.
    Example:
        >>> metric = F1Score(**)
        >>> for epoch in range(epochs):
        >>>     metric.reset()
        >>>     for batch in batchs:
        >>>         logits = model()
        >>>         metric(logits,target)
        >>>         print(metric.name(),metric.value())
    '''
    def __init__(self,thresh = 0.5, normalizate = True,task_type = 'multiclass',average = 'micro',search_thresh = False):
        super(F1Score).__init__()
        assert task_type in ['binary','multiclass']
        assert average in ['binary','micro', 'macro', 'samples', 'weighted']

        self.thresh = thresh
        self.task_type = task_type
        self.normalizate  = normalizate
        self.search_thresh = search_thresh
        self.average = average

    def thresh_search(self,y_prob):
        best_threshold = 0
        best_score = 0
        for threshold in tqdm([i * 0.01 for i in range(100)], disable=True):
            self.y_pred = y_prob > threshold
            score = self.value()
            if score > best_score:
                best_threshold = threshold
                best_score = score
        return best_threshold,best_score

    def __call__(self,logits,target):
        self.y_true = target.cpu().numpy()
        if self.normalizate and self.task_type == 'binary':
            y_prob = logits.sigmoid().data.cpu().numpy()
        elif self.normalizate and self.task_type == 'multiclass':
            y_prob = logits.softmax(-1).data.cpu().detach().numpy()
        else:
            y_prob = logits.cpu().detach().numpy()

        if self.task_type == 'binary':
            if self.thresh and self.search_thresh == False:
                self.y_pred = (y_prob > self.thresh ).astype(int)
                self.value()
            else:
                thresh,f1 = self.thresh_search(y_prob = y_prob)
                print(f"Best thresh: {thresh:.4f} - F1 Score: {f1:.4f}")

        if self.task_type == 'multiclass':
            self.y_pred = np.argmax(y_prob, 1)

    def reset(self):
        self.y_pred = 0
        self.y_true = 0

    def value(self):
        f1 = f1_score(y_true=self.y_true, y_pred=self.y_pred, average=self.average)
        return f1

    def name(self):
        return 'f1'
    
    def show(self):
        result = self.value()
        print("F1: {}".format(result))
      
        

class ClassReport(Metric):
    '''
    class report
    '''
    def __init__(self,target_names = None):
        super(ClassReport).__init__()
        self.target_names = target_names

    def reset(self):
        self.y_pred = 0
        self.y_true = 0

    def value(self):
        score = classification_report(y_true = self.y_true,
                                      y_pred = self.y_pred,
                                      target_names=self.target_names)
        print(f"\n\n classification report: {score}")

    def __call__(self,logits,target):
        _, y_pred = torch.max(logits.data, 1)
        self.y_pred = y_pred.cpu().numpy()
        self.y_true = target.cpu().numpy()

    def name(self):
        return "class_report"

class MultiLabelReport(Metric):
    '''
    multi label report
    '''
    def __init__(self,id2label = None):
        super(MultiLabelReport).__init__()
        self.id2label = id2label

    def reset(self):
        self.y_prob = 0
        self.y_true = 0

    def __call__(self,logits,target):

        self.y_prob = logits.sigmoid().data.cpu().detach().numpy()
        self.y_true = target.cpu().numpy()

    def value(self):
        for i, label in self.id2label.items():
            auc = roc_auc_score(y_score=self.y_prob[:, i], y_true=self.y_true[:, i])
            print(f"label:{label} - auc: {auc:.4f}")

    def name(self):
        return "multilabel_report"


In [10]:
## latest Metrics
x = EIM()
x(jd,predictions,true)
y = REIM()
y(jd,predictions,true)
z = RIIM()
z(jd,predictions,true)

NameError: name 'jd' is not defined

In [54]:
x.show()
y.show()
z.show()

EIM: 121.09983877356967
REIM: 23.042102879162762
RIIM: 46.366780109978585


In [55]:
## Coling Metrics 
x = EIM()
x(jd,coling,true)
y = REIM()
y(jd,coling,true)
z = RIIM()
z(jd,coling,true)
x.show()
y.show()
z.show()

EIM: 115.88814640810699
REIM: 22.51026431132325
RIIM: 61.07836631061596


In [56]:
# GalaXC MiniLM 
x = EIM()
x(jd,GAL,true)
y = REIM()
y(jd,GAL,true)
z = RIIM()
z(jd,GAL,true)
x.show()
y.show()
z.show()

EIM: 86.44667650951534
REIM: 15.590822698522494
RIIM: 25.77490234054609


# Let's analyse what happens with frequent skills in all predictions

In [57]:
f = open('skill_list (1).txt').readlines()
skill_lis = [x.split('\n')[0] for x in f]

In [58]:
from collections import defaultdict as ddict

dicti = ddict(int)
for item in true:
    for j in item:
        dicti[j]+=1
        
skill_lis_fil =[]
for key,val in dicti.items():
    if dicti[key]>20: # frequency should be more than 20
        skill_lis_fil.append(key)

    

In [59]:

lis_counts=[]

for i, (x,y) in enumerate(zip(true,GAL)):
    count =0
    for t in x:
        if t in y and t in skill_lis_fil:
            count+=1
    lis_counts.append(count)
# print(lis_counts)

lis_counts_coling=[]

for i, (x,y) in enumerate(zip(true,coling)):
    count =0
    for t in x:
        if t in y and t in skill_lis_fil:
            count+=1
    lis_counts_coling.append(count)
# print(lis_counts)

lis_counts_latest=[]

for i, (x,y) in enumerate(zip(true,lt)):
    count =0
    for t in x:
        if t in y and t in skill_lis_fil:
            count+=1
    lis_counts_latest.append(count)
# print(lis_counts)

lis_counts_fs=[]

for i, (x,y) in enumerate(zip(true,fs)):
    count =0
    for t in x:
        if t in y and t in skill_lis_fil:
            count+=1
    lis_counts_fs.append(count)
# print(lis_counts)


In [60]:
df['num_frequent_skills_predicted_galaxc_minilm'] = lis_counts
df['num_frequent_skills_predicted_latest'] = lis_counts_latest
df['num_frequent_skills_predicted_coling'] = lis_counts_coling
df['num_frequent_skills_predicted_fs'] = lis_counts_fs

In [61]:
lis_counts_t=[]

for i, x in enumerate(true):
    count =0
    for t in x:
        if t in skill_lis_fil:
            count+=1
    lis_counts_t.append(count)
    
df['num_frequent_skills_true'] = lis_counts_t
df[df.columns[7:]].head(10)

,num_frequent_skills_predicted_galaxc_minilm,num_frequent_skills_predicted_latest,num_frequent_skills_predicted_coling,num_frequent_skills_predicted_fs,num_frequent_skills_true
0,1,4,12,5,20
1,2,1,2,2,4
2,12,19,19,19,20
3,11,18,19,12,19
4,11,8,14,13,15
5,10,18,16,9,20
6,2,5,4,4,7
7,1,0,5,1,17
8,7,16,15,15,20
9,2,0,2,0,3


In [62]:
# calculate cases where less than 20 skills are predicted in coling vs latest, result is (unique, total)

def stats(lis_counts):
    count =0
    un =0
    for item in lis_counts:
        count+=item
        if item>0:
            un+=1
    return un, count
        
            


In [63]:
print(stats(lis_counts_latest))

(1983, 19733)


In [64]:
print(stats(lis_counts_coling))

(1999, 23405)


In [65]:
print(stats(lis_counts_t))

(2023, 33482)


In [66]:
print(stats(lis_counts_fs))

(1935, 14193)


In [67]:
## seems like implicit and rare skills are missing, if we keep threshold 5 we can see that sorting decresses prediction of less relevant skills.

In [68]:
print(df[df.columns[1:7]].iloc[0].to_list())

["['Project Management', 'SQL', 'Integration', 'Management', 'Software Development', 'Windows Server', 'Linux', 'Java', 'Troubleshooting', 'Business Analysis', 'Microsoft SQL Server', 'Technical Support', 'Servers', 'ITIL', 'Active Directory', 'Networking', 'Cloud Computing', 'Virtualization', 'System Administration', 'Testing']", "['Product Development', 'Manufacturing', 'Negotiation', 'Cross-functional Team Leadership', 'Solution Selling', 'Account Management', 'Sales', 'E-commerce', 'Business Development', 'New Business Development', 'Engineering', 'Digital Strategy', 'Management', 'Product Management', 'Web Analytics', 'International Sales', 'Google Analytics', 'Process Improvement', 'Marketing', 'Sales Management']", "['Management', 'Microsoft Office', 'Cloud Computing', 'Data Center', 'Integration', 'ITIL', 'Program Management', 'Project Management', 'Requirements Analysis', 'Software Development', 'Linux', 'Microsoft SQL Server', 'SQL', 'VMware', 'Unix', 'Databases', 'Servers', 

In [69]:
## Implicit vs explicit skill prediction

def imp_exp_skills(jd,prediction):
    imp_list, exp_list =[],[]
    for i,pred in enumerate(prediction):
        imp,exp =[],[]
        for item in pred:
            if item.lower() in jd[i].lower().split():
                imp.append(item)
            else:
                exp.append(item)
        imp_list.append(imp)
        exp_list.append(exp)
    return imp_list,exp_list
imp_list, exp_list = imp_exp_skills(jd,lt)
            
            

In [70]:
df['implicit_skills_pred'] = imp_list
df['explicit_skills_pred'] = exp_list

In [71]:
imp_list, exp_list = imp_exp_skills(jd,true)
df['implicit_skills_true'] = imp_list
df['explicit_skills_true'] = exp_list

imp_list, exp_list = imp_exp_skills(jd,coling)
df['implicit_skills_coling'] = imp_list
df['explicit_skills_coling'] = exp_list

In [72]:
df.columns

Index(['Job description /job requirements', 'COLING', 'GALAXC', 'True Labels',
       'GALAXC-best', 'fasttext', 'latest',
       'num_frequent_skills_predicted_galaxc_minilm',
       'num_frequent_skills_predicted_latest',
       'num_frequent_skills_predicted_coling',
       'num_frequent_skills_predicted_fs', 'num_frequent_skills_true',
       'implicit_skills_pred', 'explicit_skills_pred', 'implicit_skills_true',
       'explicit_skills_true', 'implicit_skills_coling',
       'explicit_skills_coling'],
      dtype='object')

In [73]:
df[['Job description /job requirements', 'COLING', 'GALAXC', 'True Labels','GALAXC-best', 'fasttext', 'latest','implicit_skills_pred', 'explicit_skills_pred','implicit_skills_true',
       'explicit_skills_true']].head()

,Job description /job requirements,COLING,GALAXC,True Labels,GALAXC-best,fasttext,latest,implicit_skills_pred,explicit_skills_pred,implicit_skills_true,explicit_skills_true
0,[CLS] requirements technical academic education job experience within technical function good communication written skills understanding respective market competitor products technical report writing depth knowledge respective product range strong knowledge function products fulfils project integrated solution differentiate features benefits commercial technical side present relevant recipients take ownership customer queries know didactical strategies methods train structured project management follow product related issues customers pm bu monitoring reporting repeat issues products understanding procedures processes st ensure optimal service respected counterpart regional director country sales director regional technical functions flexibility creativity determined energetic entrepreneurial thinking strong personal initiative engage improving business processes contributing ideas persuasive power assertiveness dedicated support product range team player think partnerships work collaboratively members st business customer oriented service provider attitude positive ambassador company expertise specific product group technical side drives activities support pre post sales find customer specific applications project realization country region provide second level support relevant products application includes information exchange national sales organisation nso business units bu provides application support internal external customers provides feedback using practical experience influence regional product management order improve extend line products increase product competence know technical support ts delivering technical trainings staff members well external customers support technical support product managers regional sales office rso nso solve kind technical issues prevent potential pitfalls deliver comprehensive technical trainings staff members customers increase level competence specific product range support product management defining providing feedback support development appropriate product range [SEP],"['Project Management', 'SQL', 'Integration', 'Management', 'Software Development', 'Windows Server', 'Linux', 'Java', 'Troubleshooting', 'Business Analysis', 'Microsoft SQL Server', 'Technical Support', 'Servers', 'ITIL', 'Active Directory', 'Networking', 'Cloud Computing', 'Virtualization', 'System Administration', 'Testing']","['Product Development', 'Manufacturing', 'Negotiation', 'Cross-functional Team Leadership', 'Solution Selling', 'Account Management', 'Sales', 'E-commerce', 'Business Development', 'New Business Development', 'Engineering', 'Digital Strategy', 'Management', 'Product Management', 'Web Analytics', 'International Sales', 'Google Analytics', 'Process Improvement', 'Marketing', 'Sales Management']","['Management', 'Microsoft Office', 'Cloud Computing', 'Data Center', 'Integration', 'ITIL', 'Program Management', 'Project Management', 'Requirements Analysis', 'Software Development', 'Linux', 'Microsoft SQL Server', 'SQL', 'VMware', 'Unix', 'Databases', 'Servers', 'Virtualization', 'Windows Server', 'Solution Architecture']","['Product Development', 'Manufacturing', 'Cross-functional Team Leadership', 'Project Management', 'Engineering', 'New Business Development', 'E-commerce', 'Negotiation', 'Marketing', 'Account Management', 'Solution Selling', 'Sales Management', 'Sales', 'GMP', 'Product Management', 'Business Development', 'Continuous Improvement', 'Lean Manufacturing', 'Quality Control', 'Product Marketing']","['Management', 'Manufacturing', 'Marketing', 'Testing', 'Product Development', 'Process Improvement', 'Engineering', 'Continuous Improvement', 'Linux', 'Java', 'SQL', 'Project Management', 'Supply Chain Management', 'Software Development', 'Sales', 'CRM', 'Materials Management', 'Financial Modeling', 'Auditing', 'SAP

In [74]:
df.to_csv('Job predictions.csv',index = None)

In [76]:
df[['COLING', 'GALAXC', 'True Labels','latest','implicit_skills_pred', 'implicit_skills_coling','implicit_skills_true',
       'explicit_skills_pred','explicit_skills_true']].head(10)

,COLING,GALAXC,True Labels,latest,implicit_skills_pred,implicit_skills_coling,implicit_skills_true,explicit_skills_pred,explicit_skills_true
0,"['Project Management', 'SQL', 'Integration', 'Management', 'Software Development', 'Windows Server', 'Linux', 'Java', 'Troubleshooting', 'Business Analysis', 'Microsoft SQL Server', 'Technical Support', 'Servers', 'ITIL', 'Active Directory', 'Networking', 'Cloud Computing', 'Virtualization', 'System Administration', 'Testing']","['Product Development', 'Manufacturing', 'Negotiation', 'Cross-functional Team Leadership', 'Solution Selling', 'Account Management', 'Sales', 'E-commerce', 'Business Development', 'New Business Development', 'Engineering', 'Digital Strategy', 'Management', 'Product Management', 'Web Analytics', 'International Sales', 'Google Analytics', 'Process Improvement', 'Marketing', 'Sales Management']","['Management', 'Microsoft Office', 'Cloud Computing', 'Data Center', 'Integration', 'ITIL', 'Program Management', 'Project Management', 'Requirements Analysis', 'Software Development', 'Linux', 'Microsoft SQL Server', 'SQL', 'VMware', 'Unix', 'Databases', 'Servers', 'Virtualization', 'Windows Server', 'Solution Architecture']","['Management', 'Project Management', 'Leadership', 'Strategy', 'Business Strategy', 'Strategic Planning', 'Business Development', 'Program Management', 'Marketing', 'Business Analysis', 'Sales', 'Marketing Strategy', 'Training', 'Change Management', 'Product Management', 'Analysis', 'Cross-functional Team Leadership', 'CRM', 'Integration', 'New Business Development']\n","[Management, Sales]",[Management],[Management],"[Project Management, Leadership, Strategy, Business Strategy, Strategic Planning, Business Development, Program Management, Marketing, Business Analysis, Marketing Strategy, Training, Change Management, Product Management, Analysis, Cross-functional Team Leadership, CRM, Integration, New Business Development]","[Microsoft Office, Cloud Computing, Data Center, Integration, ITIL, Program Management, Project Management, Requirements Analysis, Software Development, Linux, Microsoft SQL Server, SQL, VMware, Unix, Databases, Servers, Virtualization, Windows Server, Solution Architecture]"
1,"['Financial Analysis', 'Management', 'Portfolio Management', 'Banking', 'Investments', 'Risk Management', 'Credit', 'Finance', 'Loans', 'Asset Management', 'Commercial Banking', 'Corporate Finance', 'Retail Banking', 'Leadership', 'Credit Analysis', 'Financial Modeling', 'Capital Markets', 'Sales', 'Derivatives', 'Equities']","['Customer Service', 'Risk Management', 'Sales', 'Due Diligence', 'Negotiation', 'Real Estate', 'Logistics', 'Account Management', 'Inventory Management', 'Financial Analysis', 'Management', 'Supply Chain', 'Purchasing', 'Finance', 'Process Improvement', 'Financial Modeling', 'Procurement', 'Retail Banking', 'Mergers & Acquisitions', 'Budgets']","['Sales', 'Investments', 'Negotiation', 'Selling', 'Investment Properties', 'Property', 'Real Estate', 'Real Estate Transactions', 'Residential Homes', 'Buyer Representation', 'Condos', 'Foreclosures', 'Listings', 'Luxury', 'New Home Sales', 'Referrals', 'Relocation', 'REO', 'Short Sales', 'Single Family Homes']","['Management', 'Financial Analysis', 'Risk Management', 'Strategy', 'Financial Modeling', 'Investments', 'Business Strategy', 'Leadership', 'Finance', 'Banking', 'Business Development', 'Strategic Planning', 'Portfolio Management', 'Corporate Finance', 'Project Management', 'Analysis', 'Equities', 'Business Analysis', 'Business Planning', 'Credit']\n","[Finance, Banking]","[Banking, Finance, Sales]",[Sales],"[Management, Financial Analysis, Risk Management, Strategy, Financial Modeling, Investments, Business Strategy, Leadership, Business Development, Strategic Planning, Portfolio Management, Corporate Finance, Project Management, Analysis, Equities, Business Analysis, Business Planning, Credit]","[Investments, Negotiation, Selling, Investment Properties, Property, Real Estate,